In [ ]:
# Base de Conocimiento
KB = {
    "Hechos": [
        
    ],
    "Reglas": [
        {
            "condicion": "riesgo=bajo && eficiencia_energetica=True", 
            "decision": "Aprobado"
        },
        {
            "condicion": "riesgo=alto && impacto_ambiental=negativo",
            "decision": "Rechazado"
        },
        {
            "condicion": "endeudamiento=medio && impacto_ambiental=moderado",
            "decision": "AprobarCondicionadoGarantia"
        }
    ],
    "ReglasCombinadas": [
        {
            # DEBE SER UNA O VARIAS REGLAS???
            "condicion": "riesgo=medio && impacto_ambiental=positivo && cofinanciamiento=True", 
            "decision": "AprobadoCondicionadoMitigaciónSeguimiento"
        },
        {
            "condicion": "riesgo=bajo && impacto_ambiental=moderado && endeudamiento=medio",
            "decision": "AprobadoCondicionadoSeguimientoAmbiental"
        },
        {
            "condicion": "riesgo=medio && eficiencia_energetica=True && endeudamiento=bajo",
            "decision": "AprobadoConDescuentoTasa"
        },
        {
            "condicion": "riesgo=alto && eficiencia_energetica=False && impacto_ambiental=negativo",
            "decision": "RechazadoDefinitivo"
        },
        {
            "condicion": "riesgo=bajo && endeudamiento=alto && impacto_ambiental=positivo",
            "decision": "AprobadoCondicionadoReestructuracionDeuda"
        },
        {
            "condicion": "riesgo=alto && sector_prioritario=True && impacto_ambiental=positivo",
            "decision": "RevisionEspecializadaSectorPrioritario"
        },
    
    ]
}

In [ ]:
# Motor de inferencia
def agregar_hecho(solicitante, proyecto, condicion, valor):
    KB["Hechos"].append({
        "solicitante": solicitante,
        "proyecto": proyecto,
        "condicion": condicion,
        "valor": valor
    })

In [3]:
def hechos_de(solicitante, proyecto):
    return [h for h in KB["Hechos"] if h["solicitante"] == solicitante and h["proyecto"] == proyecto]

In [ ]:
def aplicar_reglas(solicitante, proyecto):
    hechos = hechos_de(solicitante, proyecto)
    decisiones = []
    justificaciones = []
    for regla in KB["Reglas"]:
        condiciones = regla["condicion"].split("&&")
        cumple = True
        for cond in condiciones:
            cond = cond.strip()
            if "=" in cond:
                clave, valor = cond.split("=")
                clave = clave.strip()
                valor = valor.strip()
                if valor == "True":
                    valor = True
                elif valor == "False":
                    valor = False
                encontrado = any(
                    h["condición"] == clave and h["valor"] == valor
                    for h in hechos
                )
                if not encontrado:
                    cumple = False
                    break
        if cumple:
            decisiones.append(regla["decision"])
            justificaciones.append(regla["condición"])
    return decisiones, justificaciones

In [ ]:
def priorizar_recomendaciones(decisiones):
    prioridad = {
        "Aprobado": 3,
        "AprobadoCondicionadoMitigaciónSeguimiento": 2,
        "Rechazado": 0
    }
    if not decisiones:
        return "Sin decision"
    return sorted(decisiones, key=lambda d: prioridad.get(d, -1), reverse=True)[0]

In [ ]:
def evaluar_credito(solicitante, proyecto):
    print(f"Evaluando crédito para {solicitante} y proyecto {proyecto}")
    decisiones, justificaciones = aplicar_reglas(solicitante, proyecto)
    decision_final = priorizar_recomendaciones(decisiones)
    print(f"Decision priorizada: {decision_final}")
    print(f"Justificación: {justificaciones}")
    print("---")

In [7]:
agregar_hecho("Juan", "Edificio Verde", "riesgo", "bajo")
agregar_hecho("Juan", "Edificio Verde", "eficiencia_energetica", True)

agregar_hecho("Ana", "Fabrica", "riesgo", "alto")
agregar_hecho("Ana", "Fabrica", "impacto_ambiental", "negativo")

agregar_hecho("Luis", "Planta Solar", "riesgo", "medio")
agregar_hecho("Luis", "Planta Solar", "impacto_ambiental", "positivo")
agregar_hecho("Luis", "Planta Solar", "cofinanciamiento", True)

In [8]:
evaluar_credito("Juan", "Edificio Verde")
evaluar_credito("Ana", "Fabrica")
evaluar_credito("Luis", "Planta Solar")

Evaluando crédito para Juan y proyecto Edificio Verde
Decisión priorizada: Aprobado
Justificación: ['riesgo=bajo && eficiencia_energetica=True']
---
Evaluando crédito para Ana y proyecto Fabrica
Decisión priorizada: Rechazado
Justificación: ['riesgo=alto && impacto_ambiental=negativo']
---
Evaluando crédito para Luis y proyecto Planta Solar
Decisión priorizada: AprobadoCondicionadoMitigaciónSeguimiento
Justificación: ['riesgo=medio && impacto_ambiental=positivo && cofinanciamiento=True']
---
